In [32]:
from azureml.core import Workspace
subscription_id = "15ae9cb6-95c1-483d-a0e3-b1a1a3b06324"
resource_group = "nlpinterpret"
workspace_name = "performanceBenchmarking"
workspace_region = "eastus2"

In [33]:
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")



Workspace configuration succeeded. Skip the workspace creation steps below


In [47]:
experiment_name = 'performance-benchmarking-remote-vm'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)


import os
script_folder = './sample_script'
os.makedirs(script_folder, exist_ok=True)

In [48]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpu-cluster


In [49]:
# from azureml.core.compute import ComputeTarget, RemoteCompute
# from azureml.core.compute_target import ComputeTargetException

# username = "abishkar"
# address = "52.177.60.237"

# compute_target_name = 'perfBench'
# #if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
# try:
#     attached_dsvm_compute = RemoteCompute(workspace=ws, name=compute_target_name)
#     print('found existing:', attached_dsvm_compute.name)
# except ComputeTargetException:
#     attach_config = RemoteCompute.attach_configuration(address=address,
#                                                        ssh_port=22,
#                                                        username=username,
#                                                        private_key_file='C:\\Users\\abchhetr\\.ssh\\id_rsa')
#     attached_dsvm_compute = ComputeTarget.attach(workspace=ws,
#                                                  name=compute_target_name,
#                                                  attach_configuration=attach_config)
#     attached_dsvm_compute.wait_for_completion(show_output=True)

In [50]:
from azureml.core import ScriptRunConfig
from uuid import uuid4

script_path = "test_run.py"

src = ScriptRunConfig(source_directory=script_folder, 
                      script=script_path, 
                      # pass the dataset as a parameter to the training script
                    #   arguments=['--data-folder',  
                    #              dataset.as_named_input('diabetes').as_mount('/tmp/{}'.format(uuid4()))]
                     ) 

src.run_config.framework = "python"
src.run_config.target = cpu_cluster.name


run = exp.submit(config=src)

In [51]:
from azureml.widgets import RunDetails
RunDetails(run).show()

#Don't have this to submit other experiments
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: performance-benchmarking-remote-vm_1573771032_c1519f92
Web View: https://ml.azure.com/experiments/performance-benchmarking-remote-vm/runs/performance-benchmarking-remote-vm_1573771032_c1519f92?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/nlpinterpret/workspaces/performanceBenchmarking

Streaming azureml-logs/55_azureml-execution-tvmps_d466ee5412749662e26cb675904214e4807d04e3c4fd6bc0be61d4d62af344f8_d.txt

2019-11-14T22:40:38Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_131d77e01962db5979cf4fc97d17538e
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
4cfc4741a60e: Pulling fs layer
c91e5aa79b2d: Pulling fs 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Streaming azureml-logs/75_job_post-tvmps_d466ee5412749662e26cb675904214e4807d04e3c4fd6bc0be61d4d62af344f8_d.txt

bash: /azureml-envs/azureml_1b417bb747e35859ebf611fb43071e9c/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2019-11-14T22:42:52.661662
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 163
Job release is complete. Current time:2019-11-14T22:42:54.818106

Execution Summary
RunId: performance-benchmarking-remote-vm_1573771032_c1519f92
Web View: https://ml.azure.com/experiments/performance-benchmarking-remote-vm/runs/performance-benchmarking-remote-vm_1573771032_c1519f92?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/nlpinterpret/workspaces/performanceBenchmarking

Warnings:
This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.
Please enable Docker in the e

{'runId': 'performance-benchmarking-remote-vm_1573771032_c1519f92',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2019-11-14T22:40:32.591376Z',
 'endTimeUtc': '2019-11-14T22:43:07.379489Z',
 'warnings': [{'message': "This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.\nPlease enable Docker in the environment section of your run configuration to stop seeing this warning message."}],
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': '14300bae-3303-4bb6-b0c0-329e5e6a6ad9',
  'azureml.git.repository_uri': 'https://github.com/microsoft/interpret-community-text',
  'mlflow.source.git.repoURL': 'https://github.com/microsoft/interpret-community-text',
  'azureml.git.branch': 'abishkar/performance-benchmark',
  'mlflow.source.git.branch': 'abishkar/performance-benchmark',
  'azureml.git.commit': '12b3e510b6e5651276f0b724d78ca1b4fb2517a0',
  'mlflow.source.git.commit': '12b3e510b6e5651276f0b7